This is a cover letter builder using agentic ai


In [ ]:
#imports

from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
from pydantic import BaseModel
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

: 

In [95]:
load_dotenv(override=True)


True

In [96]:
openai = OpenAI()

In [97]:
creator_model = "gpt-4o"
evaluator_model = "o4-mini"

In [98]:
with open("../about/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

with open("../about/cover_letter_template.txt", "r", encoding="utf-8") as f:
    cover_letter_template = f.read()
    

In [99]:
reader = PdfReader("../about/resume.pdf")
resume = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        resume += text

In [100]:
name = "Charles McTurland"


In [101]:
system_prompt = f"""You are a proffesional cover letter writer, and your job is to write a cover letter for {name}, highlighting {name}'s skills, experience, and achievements. 
particularly questions related to {name}'s career, background, skills and experience. 
Your responsibility is to represent {name} in the letter as faithfully as possible. 
You are given a summary of {name}'s background and Resume which you can use in the cover letter. 
You are given an example of a cover letter from {name}. Try and use a similar language and style. Do NOT include the placeholder information in the cover letter. 
Be professional and engaging, uing the tone and style suitable for a cover letter.
Do not make up any information, and only use the information provided.
Don't be too verbose, and use a 3 paragraph format.
Respond with a cover letter and nothing else.
Do not include the address or contact information. 
You will be given a job description, and you will need to tailor the cover letter to the job description.
You will be evaluated, and if evalutor decides that your cover letter is not up to standart, you will be given your previus cover letters and feedback on them. 
You have to listen to the feedback, and improve your cover letter accordingly to the feedback.
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n ## Cover Letter Template:\n{cover_letter_template}\n\n
"""



In [102]:

updated_system_prompt = system_prompt

In [103]:
evaluator_system_prompt = f"""
You are a professional evaluator that decides whether a cover letter is acceptable. 
You are provided with {name}'s summary and resume, an example of a cover letter from {name}, the job description, and the cover letter. 
Your task is to evaluate the cover letter, and reply with whether it is acceptable and your feedback. 
You need to ensure if the cover letter is professional, engaging, and tailored to the job description. 
You need to ensure if the cover letter was likely made by AI, and if it was made by AI, deny it, and provide feedback. Do not allow AI generated cover letters.
You need to ensure that the cover letter has a strong and engaging opening paragraph. 
You need to ensure that the cover letter is concise and uses the standard 3 paragraph format.
Here's the information:
\n\n## Summary:\n{summary}\n\n## Resume:\n{resume}\n\n## Cover Letter Template:\n{cover_letter_template}\n\n
With this context, please evaluate the cover letter, replying with whether the cover letter is acceptable and your feedback.
"""

In [104]:
class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [105]:
def evaluator_cover_letter(job_post, cover_letter):
    return f"""
    Here's the job posting presented by the user: \n\n{job_post}\n\n
    Here's the cover letter generated by the agent: \n\n{cover_letter}\n\n
    Please evaluate the response, replying with whether it is acceptable and your extensive feedback.
    """

In [106]:
def update_system_prompt(cover_letter, feedback):
        global updated_system_prompt
        updated_system_prompt = updated_system_prompt + f"""
        \n\n## Previous cover letter rejected\nYou just tried to create a cover letter, but the quality control rejected your cover letter\n
        ## Your attempted cover letter:\n{cover_letter}\n\n
        ## Reason for rejection:\n{feedback}\n\n
        """
        return updated_system_prompt;

In [107]:
def evaluate(job_post, cover_letter) -> Evaluation:
    messages = [
        {"role": "system", "content": evaluator_system_prompt},
        {"role": "user", "content": evaluator_cover_letter(job_post, cover_letter)},
        {"role": "user", "content": "Reply ONLY in valid JSON: {\"is_acceptable\": true/false, \"feedback\": \"...\"}"}
    ]
    response = openai.responses.parse(
        model=evaluator_model,
        reasoning = {"effort":"high"},
        input=messages,
        text_format = Evaluation,
        )
    return response.output_parsed

In [108]:
def run(prompt, job_posting):
    messages = [{"role": "system", "content": prompt}] + [{"role": "user", "content": job_posting}]
    response = openai.chat.completions.create(model=creator_model, messages=messages)
    return response.choices[0].message.content

In [109]:

def scrape_webpage_simple(url):
    try:
        # Set headers to mimic a real browser
        headers = {
            'User-Agent': 'Mozilla/5.0'
        }
        
        # Make the request
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Parse the HTML
        soup = BeautifulSoup(response.content, 'html.parser')

        # Get all text content
        return soup.get_text()
        
        
    except requests.RequestException:
        return 'error'
    except Exception:
        return 'error'


In [110]:
def requestLetter(job_posting, history):
    global updated_system_prompt
    page = scrape_webpage_simple(job_posting)
    print(page)
    if page == 'error':
        print("Failed to scrape job posting")
    else:
        job_posting = page

    cover_letter = run(system_prompt, job_posting)

    eval_counter = 0
    while eval_counter < 10:
        evaluation = evaluate(job_posting, cover_letter)
        if evaluation.is_acceptable:
            print("Passed evaluation - returning reply")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"));
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            display(Markdown(f"## Updated system prompt:\n{updated_system_prompt}"))
            updated_system_prompt = system_prompt;
            return cover_letter
        else:
            eval_counter += 1
            print("Failed evaluation - retrying")
            display(Markdown(f"## Cover Letter:\n{cover_letter}"))
            display(Markdown(f"## Feedback:\n{evaluation.feedback}"))
            cover_letter = run(update_system_prompt(cover_letter, evaluation.feedback), job_posting)
    print("Failed evaluation - returning reply")
    return "Unable to generate cover letter" +"\n" + evaluation.feedback    
    

In [111]:
gr.ChatInterface(requestLetter, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


error
Failed to scrape job posting
Failed evaluation - retrying


## Cover Letter:
[Today's Date]

[First Name] [Last Name]
Subplot.ai
New York City

Dear [First Name] [Last Name],

I am writing to express my keen interest in the AI Go-to-Market Innovator Internship at Subplot.ai, as advertised. As an aspiring software developer with a strong academic foundation in Computer Science from the University of Washington Tacoma (3.9 GPA) and hands-on experience in team leadership, I am excited about the opportunity to further explore AI-driven innovation and contribute to Subplot.ai's mission of leveraging AI to transform operations.

During my studies, I developed a strong proficiency in a variety of programming languages and frameworks, including JavaScript and Python, as demonstrated by my role as leader in multiple projects such as developing a full-stack web application and building machine learning models. These experiences not only honed my technical abilities but also reinforced my problem-solving and collaborative skills in dynamic environments. My role as a Sergeant in the US Army Reserves further developed my leadership and adaptability, qualities I am eager to bring to the collaborative and rapidly-evolving environment at Subplot.ai.

I am particularly drawn to this internship due to the emphasis on creativity and iterative learning, as well as the opportunity to engage with cutting-edge AI tools such as generative UX/UI platforms. Subplot.ai’s commitment to innovation and real-world impact resonates with my own passion for making a positive change through technology. I am excited about the prospect of contributing to your team, learning, and growing with Subplot.ai. Thank you for considering my application. I look forward to the opportunity to discuss how my background, skills, and enthusiasms align with the goals of Subplot.ai.

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
This letter reads like a generic AI output and is not sufficiently tailored to the AI Go-to-Market Innovator Internship at Subplot.ai. You left placeholder text for the hiring manager’s name, which comes across as unprofessional. While your technical background is strong, the role focuses on rapid prototyping with generative UX/UI tools, go-to-market campaigns, and sales/marketing experimentation—none of which you address with concrete examples. You need a more engaging opening that cites Subplot.ai’s specific mission (e.g. AI Vertical Agents, spreadsheet automation, entrepreneurial focus), a middle paragraph that demonstrates relevant experience in outbound outreach, digital marketing, or UX prototyping tools (v0, Lovable, Windsurf), and a closing that clearly states your enthusiasm for product-market fit experimentation and calls the reader to action. Finally, tighten the language to three concise paragraphs, remove all placeholders, and inject personality by citing why Subplot.ai specifically excites you.

Failed evaluation - retrying


...

## Feedback:
This cover letter is professional and well-structured, but it reads very much like a generic, AI-generated template and doesn’t convincingly demonstrate why you’re uniquely suited to Subplot.ai’s AI Go-to-Market Innovator role. To improve: 

• Personalize the opening. Instead of leading with broad statements, reference a specific Subplot.ai project (e.g. your interest in building outbound campaigns with AI-enhanced tools) or mention an insight you have about their product. 

• Show concrete, directly relevant experience. You note your ability to learn Docker and React quickly, but the internship is focused on prototyping UX/UI and running sales campaigns. Cite any small side project, contest, or coursework where you tried A/B testing, pitched a product, or designed a user interface—even if informal. 

• Address key responsibilities in your body paragraph: “I led an email outreach pilot in my AI class, iterated on messaging based on open rates, and increased engagement by 20%.” Or: “In our programming contest, I rapidly prototyped a user-facing dashboard using React.” 

• Demonstrate familiarity or genuine eagerness with the specific generative UX tools (v0, Lovable, Windsurf). A simple statement of intent (“I’ve started exploring Windsurf’s tutorials and plan to leverage my JavaScript background to master these platforms swiftly”) shows you’re not blank. 

• Strengthen the closing by explicitly requesting next steps (“I’d welcome the chance to discuss how I can help Subplot.ai prototype and market new AI agents this summer”). 

Finally, add a sentence or two that reflects your entrepreneurial mindset—perhaps referencing your role leading teams in Home Depot or the Army as analogous to a startup environment. Tailoring these details will make your letter feel authentic and stand out.

Failed evaluation - retrying


## Cover Letter:
[Today's Date]

Subplot.ai
New York City

Dear [Hiring Manager's Name],

When I discovered Subplot.ai's groundbreaking work with AI Vertical Agents and your dedication to transforming operational productivity, I was inspired to apply for the AI Go-to-Market Innovator Internship for Summer 2025. As a recent Computer Science graduate from the University of Washington Tacoma, I am driven by my passion for AI and my entrepreneurial spirit. I am eager to contribute to your explorative work in AI-driven productivity solutions.

My academic career equipped me with the technical and collaborative skills necessary for this role—demonstrated through projects like developing a full-stack web app and creating machine learning models, which honed my ability to rapidly adopt new technologies. While I am new to UX/UI tools such as v0, Lovable, or Windsurf, my experience with Docker and React has proven my capacity to master new platforms quickly. Further, my leadership experience as a Sergeant in the US Army Reserves has solidified my problem-solving and communication skills. These are crucial as I embark on creating and refining effective AI-driven products and executing sales campaigns using AI-enhanced strategies.

Subplot.ai's commitment to innovative AI solutions and market strategy alignment excites me. I am particularly enthused about the opportunity to work with generative AI tools to prototype products and orchestrate impactful marketing campaigns. I look forward to discussing how my background and enthusiasm can contribute to Subplot.ai's ambitious goals in AI-powered market transformation. Thank you for considering my application.

Sincerely,

Sviatoslav Rutkovskyi

## Feedback:
The cover letter is polite and correctly formatted, but it feels generic and only loosely tailored to the AI Go-to-Market Innovator role. It doesn’t demonstrate hands-on experience or passion for key responsibilities such as launching outbound campaigns, prototyping with tools like v0/Lovable/Windsurf, or analyzing market feedback. Your opening is cordial but not compelling—name the hiring manager, succinctly state why you’re uniquely drawn to Subplot.ai’s mission, and highlight your most relevant selling point in the first two sentences. In the body, replace broad statements about “passion” and “rapid adoption” with concrete examples: describe a time you led a marketing or outreach effort (even a student group or project demo) or quickly learned a new design tool. Show how your Army Reserve leadership translates into the iterative, data-driven experimentation Subplot.ai calls for. Finally, close by tying your entrepreneurial goals directly to Subplot.ai’s objectives and requesting a specific follow-up (e.g., a call or interview). Personalize each paragraph with details only you could provide—this will distinguish your letter from an AI-generated template and better align your experience with the internship’s go-to-market focus.

Failed evaluation - returning reply
